In [2]:
train_file_sent="UD_Hindi-HDTB/hi_hdtb-ud-train.txt"
train_file_stat="UD_Hindi-HDTB/hi_hdtb-ud-train.conllu" 
test_file_sent="UD_Hindi-HDTB/hi_hdtb-ud-test.txt"
test_file_stat="UD_Hindi-HDTB/hi_hdtb-ud-test.conllu" 
val_file_sent="UD_Hindi-HDTB/hi_hdtb-ud-dev.txt"
val_file_stat="UD_Hindi-HDTB/hi_hdtb-ud-dev.conllu" 

In [3]:
def file_lines(name):
    lines=[]
    with open(name,'r',encoding = 'utf-8') as f:
        lines=f.readlines()
    for i in range(0,len(lines)):
        lines[i]=lines[i].strip().split('\t')
    return lines

In [4]:
def extract_stats(line):
    stats=dict()
    stats['id']=int(line[0])
    stats['form']=line[1]
    stats['lemma']=line[2]
    stats['upos']=line[3]
    stats['xpos']=line[4]
    stats['head']=int(line[6])
    stats['deprel']=line[7]
    return stats

In [5]:
def path_exists(head,i,graph):
    while i!=head and i!=0:
        i=graph[i]
    return i==head
def projectivity(sentence):
    graph=dict()
    for word in sentence:
        word_id=word['id']
        head_id=word['head']
        graph[word_id]=head_id
    for dependent in graph:
        head=graph[dependent]
        l= min(head,dependent)+1
        r=max(head,dependent)
        for i in range(l,r,1):
            if not path_exists(head,i,graph):
                return False            
    return True

In [6]:
def get_sentences(file):
    lines=file_lines(file)
    sentences=[]
    sentence=[]
    for line in lines:
        if len(line)<10:
            if len(sentence)>0:
                #do something with sentence
                print(sentence)
                sentences.append(sentence)
                sentence=[]
        else:
            sentence.append(extract_stats(line))
    return sentences

In [51]:
from collections import defaultdict
def parse(sentence):
    buffer=list()
    stack=[0]
    arcs=list()
    states=[]
    transitions=[]
    def perform_action(action,label=''):
        nonlocal buffer,stack,arcs,states,transitions
        states.append((list(buffer),list(stack),list(arcs)))
        transitions.append((action,label))
        if action=='shift':
            stack.append(buffer.pop())
        elif action=='left_arc':
            arcs.append((stack[-1],stack[-2],label))
            stack.pop(-2)
        elif action=='right_arc':
            arcs.append((stack[-2],stack[-1],label))
            stack.pop()
    dependency_graph = defaultdict(lambda: defaultdict())
    for word in reversed(sentence): #store in reverse in buffer, so top of buffer is first word
        buffer.append(word['id'])
        dependency_graph[word['head']][word['id']]=word['deprel']
    states=[]
    transitions=[]
    head_found=dict()
    while not (len(stack)==1 and stack[0]==0 and len(buffer)==0):
        if len(stack)>=2:
            if dependency_graph.get(stack[-1]) is not None and dependency_graph[stack[-1]].get(stack[-2]) is not None:
                head_found[stack[-2]]=True
                perform_action(action='left_arc',label=dependency_graph[stack[-1]][stack[-2]])
            elif dependency_graph.get(stack[-2]) is not None and dependency_graph[stack[-2]].get(stack[-1]) is not None:
                if dependency_graph.get(stack[-1]) is not None and any([dependent not in head_found for dependent in dependency_graph[stack[-1]].keys()]):
                    perform_action(action='shift')
                else:
                    head_found[stack[-1]]=True
                    perform_action(action='right_arc',label=dependency_graph[stack[-2]][stack[-1]])
            else:
                perform_action(action='shift')
        else:
            perform_action(action='shift')
    return states,transitions

In [10]:
sentences=get_sentences(train_file_stat)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[{'id': 1, 'form': 'गांव', 'lemma': 'गांव', 'upos': 'NOUN', 'xpos': 'NN', 'head': 4, 'deprel': 'obl'}, {'id': 2, 'form': 'में', 'lemma': 'में', 'upos': 'ADP', 'xpos': 'PSP', 'head': 1, 'deprel': 'case'}, {'id': 3, 'form': 'तनाव', 'lemma': 'तनाव', 'upos': 'NOUN', 'xpos': 'NN', 'head': 4, 'deprel': 'obj'}, {'id': 4, 'form': 'बना', 'lemma': 'बन', 'upos': 'VERB', 'xpos': 'VM', 'head': 0, 'deprel': 'root'}, {'id': 5, 'form': 'हुआ', 'lemma': 'हो', 'upos': 'AUX', 'xpos': 'VAUX', 'head': 4, 'deprel': 'aux'}, {'id': 6, 'form': 'है', 'lemma': 'है', 'upos': 'AUX', 'xpos': 'VAUX', 'head': 4, 'deprel': 'aux:pass'}, {'id': 7, 'form': '।', 'lemma': '।', 'upos': 'PUNCT', 'xpos': 'SYM', 'head': 4, 'deprel': 'punct'}]
[{'id': 1, 'form': 'लोगों', 'lemma': 'लोग', 'upos': 'NOUN', 'xpos': 'NN', 'head': 10, 'deprel': 'nsubj'}, {'id': 2, 'form': 'ने', 'lemma': 'ने', 'upos': 'ADP', 'xpos': 'PSP', 'head': 1, 'deprel': 'case'}, {'id': 3, 'form': 'अंबेडकर', 'lemma': 'अंबेडकर', 'upos': 'PROPN', 'xpos': 'NNPC', 'h

In [65]:
states,transitions=parse(sentences[0])

In [66]:
for i in range(len(states)):
    print('Buffer:\t',states[i][0])
    print('Stack:\t',states[i][1])
    print('Transition:\t',transitions[i])

Buffer:	 [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
Stack:	 [0]
Transition:	 ('shift', '')
Buffer:	 [11, 10, 9, 8, 7, 6, 5, 4, 3, 2]
Stack:	 [0, 1]
Transition:	 ('shift', '')
Buffer:	 [11, 10, 9, 8, 7, 6, 5, 4, 3]
Stack:	 [0, 1, 2]
Transition:	 ('left_arc', 'det')
Buffer:	 [11, 10, 9, 8, 7, 6, 5, 4, 3]
Stack:	 [0, 2]
Transition:	 ('shift', '')
Buffer:	 [11, 10, 9, 8, 7, 6, 5, 4]
Stack:	 [0, 2, 3]
Transition:	 ('right_arc', 'case')
Buffer:	 [11, 10, 9, 8, 7, 6, 5, 4]
Stack:	 [0, 2]
Transition:	 ('shift', '')
Buffer:	 [11, 10, 9, 8, 7, 6, 5]
Stack:	 [0, 2, 4]
Transition:	 ('shift', '')
Buffer:	 [11, 10, 9, 8, 7, 6]
Stack:	 [0, 2, 4, 5]
Transition:	 ('left_arc', 'advmod')
Buffer:	 [11, 10, 9, 8, 7, 6]
Stack:	 [0, 2, 5]
Transition:	 ('shift', '')
Buffer:	 [11, 10, 9, 8, 7]
Stack:	 [0, 2, 5, 6]
Transition:	 ('left_arc', 'amod')
Buffer:	 [11, 10, 9, 8, 7]
Stack:	 [0, 2, 6]
Transition:	 ('left_arc', 'nmod')
Buffer:	 [11, 10, 9, 8, 7]
Stack:	 [0, 6]
Transition:	 ('shift', '')
Buffer:	 [11, 10, 9, 8]


In [42]:
sentences[0]

[{'id': 1,
  'form': 'यह',
  'lemma': 'यह',
  'upos': 'DET',
  'xpos': 'DEM',
  'head': 2,
  'deprel': 'det'},
 {'id': 2,
  'form': 'एशिया',
  'lemma': 'एशिया',
  'upos': 'PROPN',
  'xpos': 'NNP',
  'head': 6,
  'deprel': 'nmod'},
 {'id': 3,
  'form': 'की',
  'lemma': 'का',
  'upos': 'ADP',
  'xpos': 'PSP',
  'head': 2,
  'deprel': 'case'},
 {'id': 4,
  'form': 'सबसे',
  'lemma': 'सबसे',
  'upos': 'ADV',
  'xpos': 'INTF',
  'head': 5,
  'deprel': 'advmod'},
 {'id': 5,
  'form': 'बड़ी',
  'lemma': 'बड़ा',
  'upos': 'ADJ',
  'xpos': 'JJ',
  'head': 6,
  'deprel': 'amod'},
 {'id': 6,
  'form': 'मस्जिदों',
  'lemma': 'मस्जिद',
  'upos': 'NOUN',
  'xpos': 'NN',
  'head': 9,
  'deprel': 'nmod'},
 {'id': 7,
  'form': 'में',
  'lemma': 'में',
  'upos': 'ADP',
  'xpos': 'PSP',
  'head': 6,
  'deprel': 'case'},
 {'id': 8,
  'form': 'से',
  'lemma': 'से',
  'upos': 'ADP',
  'xpos': 'PSP',
  'head': 6,
  'deprel': 'case'},
 {'id': 9,
  'form': 'एक',
  'lemma': 'एक',
  'upos': 'NUM',
  'xpos': 'QC'

In [62]:
p=0
for sentence in sentences:
    if projectivity(sentence):
        p+=1
print(p)
print(len(sentences)-p)
print(len(sentences))

11467
1837
13304


(60, 4)